In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize  

In [3]:
df = pd.read_excel('Skills_Linkedin.xlsx')
df = df.dropna()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6025 entries, 0 to 6024
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6025 non-null   int64 
 1   job         6025 non-null   object
 2   skills      6025 non-null   object
 3   tipo        6025 non-null   object
dtypes: int64(1), object(3)
memory usage: 235.4+ KB


In [6]:
#df = df[df['cargo'] == 'Analista de Controladoria']

In [7]:
skills = df['skills']
len(skills)

6025

In [34]:
skills.replace(['\n',
                 '^.*?Expect',
                 '^.*?Qualifications',
                 '^.*?Required',
                 '^.*?expected',
                 '^.*?Responsibilities',
                 '^.*?Requisitos',
                 '^.*?Requirements',
                 '^.*?Qualificações',
                 '^.*?QualificationsRequired1',
                 '^.*?você deve ter:',
                 '^.*?experiência',
                 '^.*?você:',
                 '^.*?Desejável',
                 '^.*?great',
                 '^.*?Looking For',
                 '^.*?ll Need',
                 '^.*?Conhecimento',
                 '^.*?se:',
                 '^.*?habilidades',
                 '^.*?se:',
                 '^.*?REQUISITOS'
                 ], '', regex=True)

bad_words = ['gender', 'experience', 'application', 'Apply', 'salary', 'todos', 'os', 'company', 'identity', 'sexual', 'orientation',
            'de', 'orientação', 'sexual', 'gênero', 'committed', 'toda', 'client', 'conhecimento',
            'world', 'year', 'save', 'São', 'Paulo', 'information', 'e', 'orientação', 'sexual', 'equal', 'oppotunity', 'ambiente', 'will',
            'Experiência', 'national origin', 'todas', 'work', 'de', 'da', 'years', 'pessoa', 'clients', 'Plano', 'creating',
            'employer', 'saúde', 'em', 'working', 'pessoas', 'mais', 'data', 'people', 'dia', 'one', 'knowledges', 'plataforma',
            'ou', 'benefício', 'para', 'software', 'opportunity', 'tecnologia', 'você', 'mais', 'solution', 'national', 'origin',
            'trabalhar', 'option', 'negócio', 'empresa', 'o', 'sicence', 'team', 'é', 'veteran', 'status', 'etc', 'raça', 'cor', 'belive',
            'nossa', 'uma', 'como', 'Scientist', 'ferramenta', 'projeto', 'que', 'job', 'benefícios', 'knowledge', 'toll', 's', 'modelo',
            'desconto', 'cultura', 'serviço', 'time', 'se', 'solutions', 'mercado', 'das', 'somos', 'problema', 'mundo', 'race', 'color',
            'vaga', 'pelo', 'ser', 'show', 'Seguro', 'Se', 'um', 'Um', 'tool', 'regard', 'without', 'make', 'ao', 'técnica', 'life',
            'interested', 'diversidade', 'proud', 'ability', 'sobre', 'options', 'using', 'área', 'nosso', 'na', 'seu', 'product', 'produto',
            'building', 'skill', 'model', 'religion', 'Share', 'receive', 'consideration', 'Aqui', 'vida', 'ferramentas', 'Vale', 'Refeição',
            'Strong', 'Pay', 'range', 'available', 'part', 'trabalho', 'Alimentação', 'employment', 'qualified', 'applicants', 'gympass',
            'está', 'comprometida', 'forma', 'Transporte', 'Yes', 'gente', 'melhor', 'lugar', 'believe', 'moment', 'próximo', 'deasafio',
            'dos', 'oportunidade', 'idade', 'new', 'Try', 'Premium', 'deficiência', 'sempre', 'criar', 'employee', 'problemas', 'unavailable',
            'Brasil', 'dado', 'hiring', 'trends', 'equipe', 'recent', 'temos', 'build', 'career', 'nós', 'diferencial', 'ma',
            'total', 'oferecemos', 'contato', 'tem', 'não', 'free', 'Full','of','to','is','avançado',
            'fazer','apuração','apurar','atuação','atuar','realização','realizar','elaoração','elaborar',
            'planejamento','planejar','apoio','apoiar','facilidade','revisão','revisar','execução','executar',
            'análise','analisar','validação','validar','participação','participar','confecção','confeccionar',
            'consolidação','consolidar','desenvolvimento','desenvolver','criação','criar','auxílio','avançado','intermediário'
            'auxiliar','acompanhamento','acompanhar','controle','controlar','revisão','revisar','básico','fluente',
            'formação','formação','formar','graduação','graduar','superior','cursar','cursando','graduação','graduar',
            'vivencia','vivência','atuar','atuação','experiencia','experiência','conhecimento','conhecer','habilidade',
            'capacidade','entendimento','entender','facilidade','avançado','intermediário','intermediario','basico']

In [40]:
#stop words
stopw = 'de a o que e do da em um para é com não uma os no se na por mais as dos como mas foi ao ele das tem à seu sua ou ser quando muito há nos já está eu também só pelo pela até isso ela entre era depois sem mesmo aos ter seus quem nas me esse eles estão você tinha foram essa num nem suas meu às minha têm numa pelos elas havia seja qual será nós tenho lhe deles essas esses pelas este fosse dele tu te vocês vos lhes meus minhas teu tua teus tuas nosso nossa nossos nossas dela delas esta estes estas aquele aquela aqueles aquelas isto aquilo estou está estamos estão estive esteve estivemos estiveram estava estávamos estavam estivera estivéramos esteja estejamos estejam estivesse estivéssemos estivessem estiver estivermos estiverem hei há havemos hão houve houvemos houveram houvera houvéramos haja hajamos hajam houvesse houvéssemos houvessem houver houvermos houverem houverei houverá houveremos houverão houveria houveríamos houveriam sou somos são era éramos eram fui foi fomos foram fora fôramos seja sejamos sejam fosse fôssemos fossem for formos forem serei será seremos serão seria seríamos seriam tenho tem temos tém tinha tínhamos tinham tive teve tivemos tiveram tivera tivéramos tenha tenhamos tenham tivesse tivéssemos tivessem tiver tivermos tiverem terei terá teremos terão teria teríamos teriam'.split()

stopw = [*stopw,*bad_words]

In [41]:
def tokenize(text):
    WNLTokens = []
    chars = []
    
    WNLemmatizer = WordNetLemmatizer()
    for ch in text:
        if ch not in string.punctuation:
            chars.append(ch)
        else:
            chars.append(' ')
    txt = ''.join(chars)        
        
    tokens = word_tokenize(txt)
    for t in tokens:
        WNLTokens.append(WNLemmatizer.lemmatize(t))
    
    return WNLTokens

In [51]:
tfidf = TfidfVectorizer(max_df=0.9,min_df=3,stop_words=stopw,tokenizer=tokenize,ngram_range=(1, 2))
dtmtfidf = tfidf.fit_transform(skills)


In [52]:
%%time
LDA = LatentDirichletAllocation(n_components=1,random_state=42)
LDA.fit(dtmtfidf)

Wall time: 3.51 s


LatentDirichletAllocation(n_components=1, random_state=42)

In [53]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 20 for topic {i}')
    print([tfidf.get_feature_names()[c] for c in topic.argsort()[-30:]])
    print('\n')

Top 20 for topic 0
['atividades', 'conhecimentos', 'projetos', 'indicadores', 'completo', 'pacote office', 'ciências contábeis', 'áreas', 'ciências', 'sap', 'processos', 'tableau', 'contabilidade', 'resultados', 'pacote', 'contábil', 'sistema', 'inglês', 'power bi', 'elaboração', 'bi', 'relatórios', 'contábeis', 'análises', 'python', 'power', 'parte', 'office', 'excel', 'sql']


